В данном ноутбуке рассмотрим предобученные сети от Nvidia и от Сбера (одна и та же архитектура) для распознавания фраз на русском языке и их транскрипции в текстовую форму.

Содержание:
1. Подготовка
2. Процесс распознования
3. Получение метрик
4. Вывод

#1. Подготовка



Посмотрим на модель CPU:

In [56]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


Установка и импортирование библиотек:

In [ ]:
BRANCH = 'r1.12.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH

!pip install -q evaluate jiwer

In [ ]:
import time
import os.path as osp
import IPython.display as ipd

from evaluate import load 
import nemo.collections.asr as nemo_asr

import warnings
warnings.filterwarnings('ignore')

Примеры аудио:

In [1]:
!gdown 1BOoXkBL1FgrFO_7M_BMKmDduqij16maV
!gdown 1GSUx2b2V2iapbW79InaIqtZUsIMDgWq2

Downloading...
From: https://drive.google.com/uc?id=1BOoXkBL1FgrFO_7M_BMKmDduqij16maV
To: /content/test.wav
100% 334k/334k [00:00<00:00, 85.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GSUx2b2V2iapbW79InaIqtZUsIMDgWq2
To: /content/tes2.wav
100% 217k/217k [00:00<00:00, 83.8MB/s]


In [6]:
ipd.Audio('/content/test.wav')

In [7]:
ipd.Audio('/content/tes2.wav')

Загрузка моделей и подготовка данных:

In [ ]:
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_ru_quartznet15x5" )

In [ ]:
!wget https://sc.link/ZMv # веса для модели от Сбера

In [ ]:
golos = nemo_asr.models.EncDecCTCModel.restore_from('/content/ZMv')

In [36]:
test = [osp.join('.', '/content/test.wav')]
test2 = [osp.join('.', '/content/tes2.wav')]

#2. Процесс распознавания

In [73]:
start_time = time.time()
text = asr_model.transcribe(paths2audio_files=test)
print(f'Текст: {text[0]} \nВремя распознавания: {time.time() - start_time:.3f} секунд')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Текст: раз два три проверка связи три два раз как слышно прием 
Время распознавания: 1.403 секунд


In [74]:
start_time = time.time()
text2 = asr_model.transcribe(paths2audio_files=test2)
print(f'Текст: {text2[0]} \nВремя распознавания: {time.time() - start_time:.3f} секунд')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Текст: так нолы вдосло бабытта погровил гоговорить быстро и изжижения вметного вожны радомо в роль 
Время распознавания: 1.003 секунд


In [65]:
start_time = time.time()
text3 = golos.transcribe(paths2audio_files=test)
print(f'Текст: {text3[0]} \nВремя распознавания: {time.time() - start_time:.3f} секунд')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Текст: раз два три проверка связи три два раз как слышно прием 
Время распознавания: 1.389 секунд


In [66]:
start_time = time.time()
text4 = golos.transcribe(paths2audio_files=test2)
print(f'Текст: {text4[0]} \nВремя распознавания: {time.time() - start_time:.3f} секунд')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Текст: так новая бетестеная попытка поговорить быстро если женезнетно по важно лли тому в брове зу же прервом рали 
Время распознавания: 0.980 секунд


#3. Оценка распознавания:

Для оценки распознавания воспользуемся метрикой WER (Word Error Rate)

0 - идеальное значение

In [47]:
wer = load("wer")

In [52]:
predictions = text
references = ["раз два три проверка связи три два раз как слышно прием"]
wer_score = wer.compute(predictions=predictions, references=references)
print(f'Результат первого распознавания: {wer_score}')

Результат первого распознавания: 0.0


In [62]:
predictions = text2
references = ["так новая попытка попробую поговорить быстро и совершенно невнятно посмотрим как она справится с этой ерундой"]
wer_score = wer.compute(predictions=predictions, references=references)
print(f'Результат второго распознавания: {wer_score}')

Результат второго распознавания: 0.875


In [69]:
predictions = text3
references = ["раз два три проверка связи три два раз как слышно прием"]
wer_score = wer.compute(predictions=predictions, references=references)
print(f'Результат третьего распознавания: {wer_score}')

Результат третьего распознавания: 0.0


In [70]:
predictions = text4
references = ["так новая попытка попробую поговорить быстро и совершенно невнятно посмотрим как она справится с этой ерундой"]
wer_score = wer.compute(predictions=predictions, references=references)
print(f'Результат четвертого распознавания: {wer_score}')

Результат четвертого распознавания: 0.875


#4. Вывод

Предтренированные ASR-модели от Nvidia и Сбера одинаково хорошо справляются с медленной и четкой речью, однако для более быстрых и менее четких сэмплов результат распознавания значительно хуже у обеих моделей, которые на данный момент показывают идентичную производительность.